# 06. Support Vector Machine (SVM) 實驗

## 📖 實驗目標

在前面的實驗中，我們測試了多種模型：
- ✅ **Logistic Regression**: 高血糖/高血脂表現最佳 (AUC 0.93/0.89)
- ✅ **XGBoost**: F1 分數提升 9-39%
- ✅ **ANN**: 高血壓表現最佳 (AUC 0.803) + 最高 Recall

本 notebook 將測試 **Support Vector Machine (SVM)**，理由：
1. 📊 **高維空間分類**: 使用 kernel trick 處理非線性關係
2. 💪 **小樣本效果好**: 理論上適合 6K 樣本
3. 🎯 **泛化能力強**: 最大間隔分類器，不易過擬合
4. 🔧 **支援 class_weight**: 可處理類別不平衡

**⚠️ 注意**：
- SVM 訓練**非常慢**（6K 樣本可能需要 1-2 小時）
- 需要標準化資料（已在前面步驟完成）
- 調參困難（C, gamma 很敏感）

---

## 📋 實驗流程

1. 載入資料（使用與 03/04/05 相同的資料處理）
2. 單任務 SVM（使用 RBF kernel）
3. 與最佳模型比較
4. 結論與建議

## 1. 載入套件與資料

In [ ]:
# 基礎套件
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import time

# 機器學習套件
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix
)
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# 設定
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 套件載入完成")

In [ ]:
# 載入資料
data_path = Path('../../data/processed/SUA_CVDs_wide_format.csv')
df = pd.read_csv(data_path)

print("✅ 資料載入成功！")
print(f"資料形狀: {df.shape[0]:,} 人, {df.shape[1]} 個欄位")

## 2. 準備特徵和目標變數

In [ ]:
# 定義特徵組
demographic_features = ['sex', 'Age']
biomarker_names = ['FBG', 'TC', 'Cr', 'UA', 'GFR', 'BMI', 'SBP', 'DBP']
t1_features = [f'{name}_T1' for name in biomarker_names]
t2_features = [f'{name}_T2' for name in biomarker_names]
delta1_features = [f'Delta1_{name}' for name in biomarker_names]

# 完整特徵集
X_columns = demographic_features + t1_features + t2_features + delta1_features
X = df[X_columns]

# 目標變數（轉換為 0/1）
y_hypertension = (df['hypertension_T3'] == 2).astype(int)
y_hyperglycemia = (df['hyperglycemia_T3'] == 2).astype(int)
y_dyslipidemia = (df['dyslipidemia_T3'] == 2).astype(int)

# MTL 目標變數
y_multi = np.column_stack([y_hypertension, y_hyperglycemia, y_dyslipidemia])

print(f"特徵數: {len(X_columns)} 個")
print(f"\n目標變數分佈:")
print(f"  高血壓患病率: {y_hypertension.mean():.2%}")
print(f"  高血糖患病率: {y_hyperglycemia.mean():.2%}")
print(f"  高血脂患病率: {y_dyslipidemia.mean():.2%}")

## 3. 資料分割與標準化

⚠️ **SVM 對資料尺度非常敏感**，必須標準化！

In [ ]:
# 資料分割
X_train, X_test, y_train_multi, y_test_multi = train_test_split(
    X, y_multi, test_size=0.2, random_state=42, stratify=y_hypertension
)

# 分離三個目標變數
y_train_hp = y_train_multi[:, 0]
y_train_hg = y_train_multi[:, 1]
y_train_dl = y_train_multi[:, 2]

y_test_hp = y_test_multi[:, 0]
y_test_hg = y_test_multi[:, 1]
y_test_dl = y_test_multi[:, 2]

# 標準化（SVM 必須！）
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 資料準備完成")
print(f"訓練集: {X_train_scaled.shape[0]} 人")
print(f"測試集: {X_test_scaled.shape[0]} 人")
print(f"特徵數: {X_train_scaled.shape[1]} 個")

## 4. 單任務 SVM

使用 RBF kernel 訓練三個 SVM 模型。

**參數設定**：
- `kernel='rbf'`: 徑向基函數核（最常用）
- `C=1.0`: 正則化參數（預設值）
- `gamma='scale'`: RBF kernel 參數（自動計算）
- `class_weight='balanced'`: 處理類別不平衡
- `probability=True`: 啟用機率預測（計算 AUC 需要）

⏱️ **預計訓練時間**：每個模型 20-40 分鐘

In [ ]:
# 訓練三個單任務 SVM 模型
diseases = ['高血壓', '高血糖', '高血脂']
y_trains = [y_train_hp, y_train_hg, y_train_dl]
y_tests = [y_test_hp, y_test_hg, y_test_dl]

svm_models = {}
svm_results = []

print("======================================================================")
print("單任務 SVM 訓練中...（這會很慢，請耐心等候）")
print("======================================================================\n")

for disease, y_train, y_test in zip(diseases, y_trains, y_tests):
    print(f"訓練 {disease} 模型...")
    start_time = time.time()
    
    # 建立 SVM 模型
    model = SVC(
        kernel='rbf',
        C=1.0,
        gamma='scale',
        class_weight='balanced',
        probability=True,
        random_state=42
    )
    
    # 訓練
    model.fit(X_train_scaled, y_train)
    
    training_time = time.time() - start_time
    
    # 預測
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    # 計算指標
    auc = roc_auc_score(y_test, y_pred_proba)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    
    # 儲存結果
    svm_models[disease] = model
    svm_results.append({
        '疾病': disease,
        'AUC': auc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall,
        '訓練時間(秒)': training_time
    })
    
    # 輸出結果
    print(f"{disease}:")
    print(f"  訓練時間:  {training_time:.1f} 秒 ({training_time/60:.1f} 分鐘)")
    print(f"  AUC:       {auc:.3f}")
    print(f"  F1:        {f1:.3f}")
    print(f"  Precision: {precision:.3f}")
    print(f"  Recall:    {recall:.3f}")
    
    # 混淆矩陣
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    print(f"  混淆矩陣: TN={tn}, FP={fp}, FN={fn}, TP={tp}")
    print()

print("✅ 單任務 SVM 訓練完成！")

## 5. SVM 效能總結

In [ ]:
# 顯示 SVM 結果
df_svm = pd.DataFrame(svm_results)

print("================================================================================")
print("SVM 效能總結")
print("================================================================================")
print(df_svm.to_string(index=False))

print("\n訓練時間統計:")
print(f"  總訓練時間: {df_svm['訓練時間(秒)'].sum():.1f} 秒 ({df_svm['訓練時間(秒)'].sum()/60:.1f} 分鐘)")
print(f"  平均訓練時間: {df_svm['訓練時間(秒)'].mean():.1f} 秒")

## 6. 與最佳模型比較

In [ ]:
print("======================================================================")
print("重新訓練基準模型作為比較")
print("======================================================================\n")

# 訓練 MTL LR
mtl_lr = MultiOutputClassifier(
    LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
)
mtl_lr.fit(X_train_scaled, y_train_multi)

# 訓練 XGBoost
from xgboost import XGBClassifier

xgb_models_comparison = {}
for disease, y_train, y_test in zip(diseases, y_trains, y_tests):
    n_positive = np.sum(y_train == 1)
    n_negative = np.sum(y_train == 0)
    scale_pos_weight = n_negative / n_positive
    
    model = XGBClassifier(
        scale_pos_weight=scale_pos_weight,
        max_depth=5,
        learning_rate=0.1,
        n_estimators=100,
        random_state=42,
        eval_metric='logloss'
    )
    model.fit(X_train_scaled, y_train)
    xgb_models_comparison[disease] = model

print("✅ 基準模型訓練完成！")

In [ ]:
# 計算所有模型的結果
all_results = []

# MTL LR
y_pred_lr = mtl_lr.predict(X_test_scaled)
for i, disease in enumerate(diseases):
    y_test = y_test_multi[:, i]
    y_pred = y_pred_lr[:, i]
    y_pred_proba = mtl_lr.estimators_[i].predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'MTL LR (balanced)',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# XGBoost
for disease, y_test in zip(diseases, y_tests):
    model = xgb_models_comparison[disease]
    y_pred = model.predict(X_test_scaled)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    
    all_results.append({
        '方法': 'XGBoost (單任務)',
        '疾病': disease,
        'AUC': roc_auc_score(y_test, y_pred_proba),
        'F1': f1_score(y_test, y_pred, zero_division=0),
        'Recall': recall_score(y_test, y_pred, zero_division=0)
    })

# SVM
for result in svm_results:
    all_results.append({
        '方法': 'SVM (RBF)',
        '疾病': result['疾病'],
        'AUC': result['AUC'],
        'F1': result['F1'],
        'Recall': result['Recall']
    })

df_all = pd.DataFrame(all_results)

print("================================================================================")
print("所有模型效能比較 (含 SVM)")
print("================================================================================")
print(df_all[['方法', '疾病', 'AUC', 'F1', 'Recall']].to_string(index=False))

## 7. 視覺化比較

In [ ]:
# 建立視覺化
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for idx, metric in enumerate(['AUC', 'F1', 'Recall']):
    ax = axes[idx]
    
    # 準備資料
    pivot_data = df_all.pivot(index='疾病', columns='方法', values=metric)
    
    # 繪圖
    pivot_data.plot(kind='bar', ax=ax, width=0.8)
    ax.set_title(f'{metric} 比較', fontsize=14, fontweight='bold')
    ax.set_ylabel(metric, fontsize=12)
    ax.set_xlabel('疾病', fontsize=12)
    ax.legend(title='方法', fontsize=9, loc='upper right')
    ax.grid(axis='y', alpha=0.3)
    ax.set_ylim(0, 1)
    
    # 添加數值標籤
    for container in ax.containers:
        ax.bar_label(container, fmt='%.2f', fontsize=8)

plt.tight_layout()
plt.savefig('../../docs/experiments/svm_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ 視覺化完成，已儲存至 docs/experiments/svm_comparison.png")

## 8. 結論

### 🎯 SVM 效能總結

本實驗測試了 Support Vector Machine (SVM) 在三高疾病預測上的表現。

### 📊 主要發現

**根據實際執行結果填寫**：

1. **SVM vs 其他模型**
   - SVM 的 AUC 表現：[填寫實際結果]
   - 與 XGBoost 相比：[填寫比較結果]
   - 與 LR 相比：[填寫比較結果]
   - 與 ANN 相比：[填寫比較結果]

2. **訓練時間**
   - 總訓練時間：[填寫實際時間]
   - 與其他模型比較：SVM >> ANN > XGBoost > LR（慢很多）

3. **SVM 的優缺點**
   - ✅ 優點：kernel trick、理論保證、泛化能力
   - ❌ 缺點：訓練超慢、調參困難、可解釋性差、記憶體需求大

### 💡 實務建議

1. **何時選擇 SVM？**
   - 小資料集（<1000 樣本）
   - 高維度特徵空間
   - 不在意訓練時間
   - 需要理論保證

2. **何時不選擇 SVM？**
   - 大資料集（>5000 樣本）→ 用 XGBoost 或 ANN
   - 需要快速訓練 → 用 LR
   - 需要可解釋性 → 用 LR 或 XGBoost
   - 需要線上學習 → 用 LR

### 📝 最終模型排名（根據實際結果）

**高血壓**：
- 最佳 AUC: [填寫]
- 最佳 F1: [填寫]
- 推薦模型: [填寫]

**高血糖**：
- 最佳 AUC: [填寫]
- 最佳 F1: [填寫]
- 推薦模型: [填寫]

**高血脂**：
- 最佳 AUC: [填寫]
- 最佳 F1: [填寫]
- 推薦模型: [填寫]

### 🎓 總結

SVM 在本研究中的表現 [根據結果填寫]，考慮到其訓練時間和複雜度，[建議/不建議] 在實務上使用。

對於 6K 樣本的三高預測任務，**推薦使用**：
- 高血壓預測: [根據結果推薦]
- 高血糖預測: [根據結果推薦]
- 高血脂預測: [根據結果推薦]